In [15]:
from csrank.dataset_reader import SyntheticDatasetGenerator
from sklearn.utils import check_random_state
from sklearn.model_selection import ShuffleSplit
from csrank.fate_ranking import FATEObjectRanker
from csrank.callbacks import DebugOutput
from csrank.tuning import ParameterOptimizer
import logging
from csrank.util import configure_logging_numpy_keras
import numpy as np
from keras import backend as K
import os

Function to convert the duration in string to the time required in microseconds

In [16]:
import re
def get_duration_microsecond(duration):
    time = int(re.findall(r'\d+', duration)[0])
    d = duration.split(str(time))[1].upper()
    options = {"D": 24 * 60 * 60 * 1e6, "H": 60 * 60 * 1e6, "M": 60 * 1e6}
    return options[d] * time

Configuring the keras and tensorflow. Defining the parameters for dataset reader. Defining the splits for the optimizer.

In [17]:
logger = configure_logging_numpy_keras(log_path=os.path.join(os.getcwd(), 'logs' ,"test_models.log"), name='Experiment')
n_features = 2
n_instances = 10000
n_objects = 5
random_state = check_random_state(42)
skf = ShuffleSplit(n_splits=2, test_size=0.5, random_state=random_state)
epochs = 100

Defining Ranker init and fitting parameters.
Defining the Parameter optimizer init and fitting parameters

In [18]:
ranker_params = {'n_objects': n_objects,'n_features':n_features, 'n_object_features':n_features}
fit_params = {'epochs': epochs,'log_callbacks':[DebugOutput()]}
medoids_params = {'dataset_type':"medoid",'n_test_instances': n_instances, 'n_train_instances': n_instances,'n_features': n_features, 'n_objects': n_objects,
                      'random_state': random_state}
optimizer_fit_params = {'n_iter': 1, 'cv_iter': skf, 'optimizer': None, "total_duration":get_duration_microsecond("10h")}

optimizer_params = {'ranker_class': FATEObjectRanker, 'fit_params': fit_params, 'ranker_params': ranker_params,
                        'random_state': random_state, "optimizer_path":os.path.join(os.getcwd(), 'logs',"optimizer")}


Generating the dataset

In [19]:
dr = SyntheticDatasetGenerator(**medoids_params)
X,Y,X_test,Y_test = dr.get_single_train_test_split()

Fitting the optimizer

In [ ]:
optimizer_model = ParameterOptimizer(**optimizer_params)
optimizer_model.fit(X, Y, **optimizer_fit_params)

Evaluate the Ranker with best parameters found by the optimizer on the test dataset

In [14]:
if optimizer_model.model is None:
    optimizer_model.model = optimizer_model._ranker_class(random_state=optimizer_model.random_state,
                                                          **optimizer_model._ranker_params)
    best_point = optimizer_model.optimizer.Xi[np.argmin(optimizer_model.optimizer.yi)]
    optimizer_model.model.set_tunable_parameters(best_point)
    logger.info(optimizer_model.model.__dict__)
    optimizer_model.model.fit(X_train, Y_train, **optimizer_model._fit_params)
predicted = optimizer_model.predict_scores(test)

NameError: name 'np' is not defined